## Lab 2

### Part 3. Poetry generation

Let's try to generate some poetry using RNNs. 

You have several choices here: 

* The Shakespeare sonnets, file `sonnets.txt` available in the notebook directory.

* Роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина. В предобработанном виде доступен по [ссылке](https://github.com/attatrol/data_sources/blob/master/onegin.txt).

* Some other text source, if it will be approved by the course staff.

Text generation can be designed in several steps:
    
1. Data loading.
2. Dictionary generation.
3. Data preprocessing.
4. Model (neural network) training.
5. Text generation (model evaluation).


In [ ]:
import string
import os
import numpy as np


### Data loading: Shakespeare

Shakespeare sonnets are awailable at this [link](http://www.gutenberg.org/ebooks/1041?msg=welcome_stranger). In addition, they are stored in the same directory as this notebook (`sonnetes.txt`). Simple preprocessing is already done for you in the next cell: all technical info is dropped.

In [ ]:
if not os.path.exists('sonnets.txt'):
    !wget https://raw.githubusercontent.com/girafe-ai/ml-mipt/master/homeworks_basic/Lab2_DL/sonnets.txt

with open('sonnets.txt', 'r') as iofile:
    text = iofile.readlines()
    
TEXT_START = 45
TEXT_END = -368
text = text[TEXT_START : TEXT_END]
assert len(text) == 2616

In opposite to the in-class practice, this time we want to predict complex text. Let's reduce the complexity of the task and lowercase all the symbols.

Now variable `text` is a list of strings. Join all the strings into one and lowercase it.

In [ ]:
# Join all the strings into one and lowercase it
# Put result into variable text.

# Your great code here
text = ''.join(text).lower()
assert len(text) == 100225, 'Are you sure you have concatenated all the strings?'
assert not any([x in set(text) for x in string.ascii_uppercase]), 'Uppercase letters are present'
print('OK!')

OK!


### Data loading: "Евгений Онегин"


In [ ]:
!wget https://raw.githubusercontent.com/attatrol/data_sources/master/onegin.txt
    
with open('onegin.txt', 'r') as iofile:
    text = iofile.readlines()
    
text = [x.replace('\t\t', '') for x in text]

--2021-06-11 20:53:10--  https://raw.githubusercontent.com/attatrol/data_sources/master/onegin.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262521 (256K) [text/plain]
Saving to: ‘onegin.txt’

onegin.txt          100%[===================>] 256.37K  --.-KB/s    in 0.006s  

2021-06-11 20:53:11 (44.3 MB/s) - ‘onegin.txt’ saved [262521/262521]



In opposite to the in-class practice, this time we want to predict complex text. Let's reduce the complexity of the task and lowercase all the symbols.

Now variable `text` is a list of strings. Join all the strings into one and lowercase it.

In [ ]:
# Join all the strings into one and lowercase it
# Put result into variable text.

# Your great code here


In [ ]:
type(text)

str

Put all the characters, that you've seen in the text, into variable `tokens`.

In [ ]:
tokens = sorted(set(text + '$@&'))

Create dictionary `token_to_idx = {<char>: <index>}` and dictionary `idx_to_token = {<index>: <char>}`

In [ ]:
token_to_idx = {token: idx for idx, token in enumerate(tokens)}
idx_to_token = {idx: token for idx, token in enumerate(tokens)}
assert len(token_to_idx) == len(idx_to_token)


*Comment: in this task we have only 38 different tokens, so let's use one-hot encoding.*

### Building the model

Now we want to build and train recurrent neural net which would be able to something similar to Shakespeare's poetry.

Let's use vanilla RNN, similar to the one created during the lesson.

In [72]:
def to_matrix(pieces, max_len=None):
  if max_len == None: 
    max_len = max(map(len, pieces))
  pad = token_to_idx['@']
  sos = token_to_idx['$']
  eos = token_to_idx['&']

  text_ix = np.zeros((len(pieces), max_len), dtype='int32') + pad
  #text_ix[:0] += sos

  for i in range(len(pieces)): 
    line_ix = [token_to_idx[c] for c in pieces[i]]
    #line_ix.append(eos)
    text_ix[i, :len(line_ix)] = line_ix
    
  return text_ix

In [ ]:
tokens

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 ':',
 ';',
 '?',
 '@',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [73]:
names = ['olga', 'maxim', 'catherine', 'mary']
to_matrix(names)

array([[29, 26, 21, 15, 14, 14, 14, 14, 14],
       [27, 15, 38, 23, 27, 14, 14, 14, 14],
       [17, 15, 34, 22, 19, 32, 23, 28, 19],
       [27, 15, 32, 39, 14, 14, 14, 14, 14]], dtype=int32)

In [ ]:
import torch, torch.nn as nn
import torch.nn.functional as F

In [ ]:
class CharRNNCell(nn.Module):
    """
    Implement the scheme above as torch module
    """
    def __init__(self, num_tokens=len(tokens), embedding_size=16, rnn_num_units=64):
        super(self.__class__,self).__init__()
        self.num_units = rnn_num_units
        
        self.embedding = nn.Embedding(num_tokens, embedding_size)
        self.rnn_update = nn.Linear(embedding_size + rnn_num_units, rnn_num_units)
        self.rnn_to_logits = nn.Linear(rnn_num_units, num_tokens)
        
    def forward(self, x, h_prev):
        """
        This method computes h_next(x, h_prev) and log P(x_next | h_next)
        We'll call it repeatedly to produce the whole sequence.
        
        :param x: batch of character ids, containing vector of int64
        :param h_prev: previous rnn hidden states, containing matrix [batch, rnn_num_units] of float32
        """
        # get vector embedding of x
        x_emb = self.embedding(x)
        
        # compute next hidden state using self.rnn_update
        # hint: use torch.cat(..., dim=...) for concatenation
        x_and_h = torch.cat([x_emb, h_prev], dim=1)
        h_next = self.rnn_update(x_and_h)
        
        h_next = torch.tanh(h_next)
        
        assert h_next.size() == h_prev.size()
        
        #compute logits for next character probs
        logits = self.rnn_to_logits(h_next)
        
        return h_next, F.log_softmax(logits, -1)
    
    def initial_state(self, batch_size):
        """ return rnn state before it processes first input (aka h0) """
        return torch.zeros(batch_size, self.num_units, requires_grad=True)

In [ ]:
char_rnn = CharRNNCell()
criterion = nn.NLLLoss()

In [ ]:
def rnn_loop(char_rnn, batch_ix):
    """
    Computes log P(next_character) for all time-steps in names_ix
    :param names_ix: an int32 matrix of shape [batch, time], output of to_matrix(names)
    """
    batch_size, max_length = batch_ix.size()
    hid_state = char_rnn.initial_state(batch_size)
    logprobs = []

    for x_t in batch_ix.transpose(0,1):
        hid_state, logp_next = char_rnn(x_t, hid_state)  # <-- here we call your one-step code
        logprobs.append(logp_next)
        
    return torch.stack(logprobs, dim=1)

In [ ]:
pieces = [text[i:i+30] for i in range(300)]

In [ ]:
pieces

['  from fairest creatures we de',
 ' from fairest creatures we des',
 'from fairest creatures we desi',
 'rom fairest creatures we desir',
 'om fairest creatures we desire',
 'm fairest creatures we desire ',
 ' fairest creatures we desire i',
 'fairest creatures we desire in',
 'airest creatures we desire inc',
 'irest creatures we desire incr',
 'rest creatures we desire incre',
 'est creatures we desire increa',
 'st creatures we desire increas',
 't creatures we desire increase',
 ' creatures we desire increase,',
 'creatures we desire increase,\n',
 'reatures we desire increase,\n ',
 'eatures we desire increase,\n  ',
 'atures we desire increase,\n  t',
 'tures we desire increase,\n  th',
 'ures we desire increase,\n  tha',
 'res we desire increase,\n  that',
 'es we desire increase,\n  that ',
 's we desire increase,\n  that t',
 ' we desire increase,\n  that th',
 'we desire increase,\n  that the',
 'e desire increase,\n  that ther',
 ' desire increase,\n  that there',
 'desir

In [ ]:
batch_ix = to_matrix(pieces)
batch_ix = torch.tensor(batch_ix, dtype=torch.int64)

logp_seq = rnn_loop(char_rnn, batch_ix)

assert torch.max(logp_seq).data.numpy() <= 0
assert tuple(logp_seq.size()) ==  batch_ix.shape + (len(tokens),)

In [ ]:
predictions_logp = logp_seq[:, :-1]
actual_next_tokens = batch_ix[:, 1:]

logp_next = torch.gather(predictions_logp, dim=2, index=actual_next_tokens[:,:,None])

loss = -logp_next.mean()
loss.backward()

In [ ]:
import random

In [74]:
def get_pieces(piece_len, num):
  res = []
  for i in range(num):
    start = random.randrange(0, len(text) - piece_len)
    res.append(text[start:start + piece_len])
  return res


In [75]:
get_pieces(20, 5)

['t.\n\n  cxi\n\n  o! for ',
 'he treasure of his s',
 'n to my sportive blo',
 's that to thee resor',
 'on myself with prese']

In [76]:
from IPython.display import clear_output
from random import sample

char_rnn = CharRNNCell()
opt = torch.optim.Adam(char_rnn.parameters())
history = []

In [ ]:
MAX_LEN = 200
pieces = get_pieces(MAX_LEN, 1000)

actual_next_tokens = batch_ix
for i in range(1000):
  samples = torch.tensor(to_matrix(pieces), dtype=torch.int64)
  batch_ix = samples[:, :100]
  opt.zero_grad()
  logp_seq = rnn_loop(char_rnn, batch_ix) 
  actual_next_tokens = samples[:, 100:]
  predictions_logp = logp_seq
  logp_next = torch.gather(predictions_logp, dim=2, index=actual_next_tokens[:,:,None])
  loss = -logp_next.mean()
  print(loss)
  loss.backward()
  opt.step()





tensor(3.6253, grad_fn=<NegBackward>)
tensor(3.6052, grad_fn=<NegBackward>)
tensor(3.5821, grad_fn=<NegBackward>)
tensor(3.5551, grad_fn=<NegBackward>)
tensor(3.5234, grad_fn=<NegBackward>)
tensor(3.4858, grad_fn=<NegBackward>)
tensor(3.4416, grad_fn=<NegBackward>)
tensor(3.3908, grad_fn=<NegBackward>)
tensor(3.3349, grad_fn=<NegBackward>)
tensor(3.2773, grad_fn=<NegBackward>)
tensor(3.2225, grad_fn=<NegBackward>)
tensor(3.1748, grad_fn=<NegBackward>)
tensor(3.1359, grad_fn=<NegBackward>)
tensor(3.1058, grad_fn=<NegBackward>)
tensor(3.0829, grad_fn=<NegBackward>)
tensor(3.0656, grad_fn=<NegBackward>)
tensor(3.0526, grad_fn=<NegBackward>)
tensor(3.0426, grad_fn=<NegBackward>)
tensor(3.0349, grad_fn=<NegBackward>)
tensor(3.0302, grad_fn=<NegBackward>)
tensor(3.0279, grad_fn=<NegBackward>)
tensor(3.0296, grad_fn=<NegBackward>)
tensor(3.0251, grad_fn=<NegBackward>)
tensor(3.0206, grad_fn=<NegBackward>)
tensor(3.0162, grad_fn=<NegBackward>)
tensor(3.0120, grad_fn=<NegBackward>)
tensor(3.008

In [ ]:
print(to_matrix(text[::100]))

[[14  1  4]
 [14 34  4]
 [14  1  4]
 ...
 [14 32  4]
 [14 23  4]
 [14 32  4]]


In [ ]:
MAX_LENGTH = 500

Plot the loss function (axis X: number of epochs, axis Y: loss function).

In [ ]:
# Your plot code here

In [ ]:
def generate_sample(char_rnn, seed_phrase=' Hello', max_length=MAX_LENGTH, temperature=1.0):
    '''
    ### Disclaimer: this is an example function for text generation.
    ### You can either adapt it in your code or create your own function
    
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs, 
        smaller temperature converges to the single most likely output.
        
    Be careful with the model output. This model waits logits (not probabilities/log-probabilities)
    of the next symbol.
    '''
    
    x_sequence = [token_to_id[token] for token in seed_phrase]
    x_sequence = torch.tensor([[x_sequence]], dtype=torch.int64)
    hid_state = char_rnn.initial_state(batch_size=1)
    
    #feed the seed phrase, if any
    for i in range(len(seed_phrase) - 1):
        print(x_sequence[:, -1].shape, hid_state.shape)
        out, hid_state = char_rnn(x_sequence[:, i], hid_state)
    
    #start generating
    for _ in range(max_length - len(seed_phrase)):
        print(x_sequence.shape, x_sequence, hid_state.shape)
        out, hid_state = char_rnn(x_sequence[:, -1], hid_state)
        # Be really careful here with the model output
        p_next = F.softmax(out / temperature, dim=-1).data.numpy()[0]
        
        # sample next token and push it back into x_sequence
        print(p_next.shape, len(tokens))
        next_ix = np.random.choice(len(tokens), p=p_next)
        next_ix = torch.tensor([[next_ix]], dtype=torch.int64)
        print(x_sequence.shape, next_ix.shape)
        x_sequence = torch.cat([x_sequence, next_ix], dim=1)
        
    return ''.join([tokens[ix] for ix in x_sequence.data.numpy()[0]])

In [ ]:
# An example of generated text.
print(generate_sample(max_length=500, temperature=0.2))

TypeError: ignored

### More poetic model

Let's use LSTM instead of vanilla RNN and compare the results.

Plot the loss function of the number of epochs. Does the final loss become better?

In [ ]:
# Your beautiful code here

Generate text using the trained net with different `temperature` parameter: `[0.1, 0.2, 0.5, 1.0, 2.0]`.

Evaluate the results visually, try to interpret them.

In [ ]:
# Text generation with different temperature values here

### Saving and loading models

Save the model to the disk, then load it and generate text. Examples are available [here](https://pytorch.org/tutorials/beginner/saving_loading_models.html]).

In [ ]:
# Saving and loading code here

### References
1. <a href='http://karpathy.github.io/2015/05/21/rnn-effectiveness/'> Andrew Karpathy blog post about RNN. </a> 
There are several examples of genration: Shakespeare texts, Latex formulas, Linux Sourse Code and children names.
2. <a href='https://github.com/karpathy/char-rnn'> Repo with char-rnn code </a>
3. Cool repo with PyTorch examples: [link](https://github.com/spro/practical-pytorch`)